In [ ]:
%pip install langchain -q
%pip install -qU transformers accelerate einops langchain xformers bitsandbytes
%pip install accelerate

In [ ]:
from translate import Translator
from torch import cuda, bfloat16
import transformers
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline

In [ ]:
# Function To Translate from Telugu to English
def translate_telugu_to_english(text):
    translator= Translator(to_lang="en", from_lang="te")
    translation = translator.translate(text)
    return translation

In [ ]:
# Function To Translate from English to Telugu
def translate_english_to_telugu(text):
    translator= Translator(to_lang="te", from_lang="en")
    translation = translator.translate(text)
    return translation

In [ ]:
# Model used at the core of the conversation chain is Facebook BlenderBot-400M
model_name = 'facebook/blenderbot-400M-distill'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
# set quantization configuration to load large model with less GPU memory
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

tokenizer=transformers.AutoTokenizer.from_pretrained(model_name)

model = transformers.AutoModelForSeq2SeqLM.from_pretrained(
    model_name
)
model.eval()
print(f"Model loaded on {device}")

In [ ]:
# Defining the Hugging Face Pipeline for responding to text messages, what tokeniser must be used and all are also specified, this pipeline will act as the LLM in LangChain's conversation Chain
generate_text = transformers.pipeline(
    task="text2text-generation",
    tokenizer=tokenizer,
    model=model
)

In [ ]:
# Prompt template defined using LangChain which will be determining the behavior of the bot
template = """You are a generic friendly chatbot that answers well and respectfully to humans
{question}
"""
prompt = PromptTemplate(template=template, input_variables=['question'])
llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
# Defining the nature of Conversation chain using LangChain's LLMChain feature
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose= True)

In [ ]:
# Conversation loop
while (True):
  dialogue = input("user: ")

  if dialogue == "వెళ్ళొస్తాను" or dialogue == "ఇక సెలవు" : # Telugu equivalents of Goodbye
    print("Chatbot : సంభాషణ ముగిసింది, ధన్యవాదాలు") # Chatbot breaks loop saying "Conversation ended, thank you"
    break

  english_translation = translate_telugu_to_english(str(dialogue))
  print(english_translation, "\n")

  response = llm_chain.run(english_translation)
  print(response, "\n")

  output = (translate_english_to_telugu(response))

  print("Chatbot : ", output, "\n")